### Reading data from the silver --> Gold


In [0]:
df=spark.read.table("workspace.raw_data_taxi.silver_taxi_data")

# Defining star schema

### Dimension of date

In [0]:
from pyspark.sql.functions import col, date_format, month, year, dayofweek,unix_date


dim_date = df.select("trip_date").distinct() \
     .withColumn("date_key", unix_date(col("trip_date"))) \
    .withColumn("day_of_week", date_format(col("trip_date"), "E")) \
    .withColumn("month", month(col("trip_date"))) \
    .withColumn("year", year(col("trip_date"))) \
    .withColumn("is_weekend", (dayofweek(col("trip_date")) >= 6).cast("boolean"))

### Dimension table of vendorID

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

dim_vendor = df.select("vendorID").distinct() \
    .withColumn("vendor_key", monotonically_increasing_id())

### Dimension table of Payments

In [0]:
dim_payment = df.select(
    col("payment_type").alias("payment_type_code"),
    col("payment_type_normalized")
).distinct() \
.withColumn("payment_key", monotonically_increasing_id())

### Dimension table of locations

In [0]:
dim_location = df.select(
    col("PULocationID").alias("LocationID")
).union(
    df.select(col("DOLocationID").alias("LocationID"))
).distinct() \
.withColumn("location_key", monotonically_increasing_id())

### Joining fact tables and dimension tables

In [0]:
from pyspark.sql.functions import col

# Alias each DataFrame
df_base = df.alias("f")
dim_date_a = dim_date.alias("d")
dim_vendor_a = dim_vendor.alias("v")
dim_payment_a = dim_payment.alias("p")
dim_location_a = dim_location.alias("l")

# Build fact table with qualified column references
fact_trips = df_base \
    .join(dim_date_a, col("f.trip_date") == col("d.trip_date"), "left") \
    .join(dim_vendor_a, col("f.vendorID") == col("v.vendorID"), "left") \
    .join(dim_payment_a, col("f.payment_type_normalized") == col("p.payment_type_normalized"), "left") \
    .join(dim_location_a, col("f.PULocationID") == col("l.LocationID"), "left") \
    .select(
        col("f.trip_id"),
        col("d.date_key"),
        col("v.vendor_key"),
        col("p.payment_key"),
        col("l.location_key").alias("pickup_location_key"),
        col("f.trip_distance"),
        col("f.trip_duration_minutes"),
        col("f.passenger_count"),
        col("f.fare_amount"),
        col("f.tip_amount"),
        col("f.total_amount")
    )

###  Save latest_ts into a metadata table for incremental load

In [0]:
latest_ts = df_base.selectExpr("max(tpep_pickup_datetime)").collect()[0][0]
# Save latest_ts into a metadata table for incremental loads

In [0]:
fact_trips = fact_trips.dropDuplicates(["trip_id"])

In [0]:
fact_trips.write.mode("overwrite").format("delta").partitionBy("date_key").saveAsTable("gold.fact_trips")
dim_date.write.mode("overwrite").format("delta").saveAsTable("gold.dim_date")
dim_vendor.write.mode("overwrite").format("delta").saveAsTable("gold.dim_vendor")
dim_payment.write.mode("overwrite").format("delta").saveAsTable("gold.dim_payment")
dim_location.write.mode("overwrite").format("delta").saveAsTable("gold.dim_location")

### Merging the latest data

In [0]:
from delta.tables import DeltaTable

fact_table = DeltaTable.forName(spark, "gold.fact_trips")

fact_table.alias("t").merge(
    fact_trips.alias("s"),
    "t.trip_id = s.trip_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

### Overwrite the fact and dimension table

In [0]:
%sql
select * from gold.dim_date

trip_date,date_key,day_of_week,month,year,is_weekend
2020-12-31,18627,Thu,12,2020,false
2021-01-01,18628,Fri,1,2021,true


In [0]:
%sql
select * from gold.fact_trips

trip_id,date_key,vendor_key,payment_key,pickup_location_key,trip_distance,trip_duration_minutes,passenger_count,fare_amount,tip_amount,total_amount
008c503251cfd929305b21e4698fca64b3b59ed966e72fadb62e26d2266a9f46,18628,0,3,10,3.6,18.87,1,15.0,4.7,23.5
00abdead617377b6d2d2aa53c7c86c38bfdc4def6093131e6d066a655d5b7e10,18628,0,3,58,1.21,7.15,2,7.0,2.49,10.79
00e73106c67c2bac2d125abf1a45ef039257636c5295eb48d4885512820f3a35,18628,1,3,46,4.1,14.43,1,14.5,5.45,23.75
00f7629efe20dd3684449e4d278ba7e558b6aa11e4369a45866e93c72ac22f11,18628,0,3,39,1.99,12.47,1,10.0,2.76,16.56
0171352ea7422396adbf923b69700fbbeb7f6b98cd544ef8a5193acc71aadcf3,18628,0,3,59,3.79,12.48,2,13.0,5.04,21.84
0211e2d4bf05ede820868d170c8a14e01ad235b263a93eb24b201702663ec933,18628,0,2,16,1.96,7.25,2,8.0,0,11.8
02244aed8ff0ac97fdf91872425216aeb251212f88426c1c5d7f6240fdf19559,18628,0,3,21,0.74,3.73,1,5.0,0,8.8
029b4d924846f8df8369207b8c9b3c17e90b4b721c5830060e42845aa5803d16,18628,0,3,10,4.17,15.65,1,15.0,0,18.8
02e8545cc8245a7ad0da72ea1a3089f3c76aab6a17f647c5b0050e1321dc8fbd,18628,1,2,62,1.2,6.82,1,7.0,0,8.3
034d885644be9a2988af32f4b958e4092db7b0f6b490dac39bf3ab4fcb80830d,18628,0,3,72,3.06,11.87,1,11.5,1,16.3
